In [2]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 4.4 MB/s eta 0:00:00


In [1]:
import psycopg2
import pandas as pd

In [6]:
def db_creation():
    #connect to postgres
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=password")
    #create cursor
    cur = conn.cursor()
    #automatic commit after commands
    conn.set_session(autocommit=True)
    
    
    #create carsales database
    cur.execute("DROP DATABASE carsalesdb")
    cur.execute("CREATE DATABASE carsalesdb")
    
    #close connection to postgres default db
    conn.close()

    #connect to carsalesdb
    conn = psycopg2.connect("host=localhost dbname=carsalesdb user=postgres password=password")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    return cur, conn

In [8]:
cur, conn = db_creation()

In [9]:
#car table creation
car_table = ("""CREATE TABLE IF NOT EXISTS cars(
transaction_id INT PRIMARY KEY,
car_make VARCHAR,
car_model VARCHAR,
car_year INT)""")
cur.execute(car_table)
conn.commit()

In [10]:
#transaction table creation
transaction_table = ("""CREATE TABLE IF NOT EXISTS transactions(
transaction_id INT PRIMARY KEY,
date DATE,
salesperson VARCHAR,
customer VARCHAR)""")
cur.execute(transaction_table)
conn.commit()

In [11]:
#sales table creation
sales_table = ("""CREATE TABLE IF NOT EXISTS sales(
salesperson VARCHAR,
sales_price INT,
commission_rate FLOAT,
commission_earned FLOAT)""")
cur.execute(sales_table)
conn.commit()

In [2]:
#data load
Data = pd.read_csv("car_sales_data.csv")

In [28]:
#cleaning data and creating datasets to put values into tables
Cars_data = Data[['Transaction_id', 'Car_Make', 'Car_Model', 'Car_Year']]
Transactions_data = Data[['Transaction_id', 'Date', 'Salesperson', 'Customer']]
Sales_data = Data[['Salesperson', 'Sales_Price', 'Commission_Rate', 'Commission_Earned']]

In [26]:
#insert data into tables
car_table_insert = ("""INSERT INTO cars(
transaction_id,
car_make,
car_model,
car_year)
VALUES (%s, %s, %s, %s)
""")

In [27]:
for i, row in Cars_data.iterrows():
    cur.execute(car_table_insert, list(row))
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "cars_pkey"
DETAIL:  Key (transaction_id)=(1) already exists.


In [30]:
with open(Transactions_data, 'r') as f:
    # Notice that we don't need the csv module.
    next(f) # Skip the header row.
    cur.copy_from(f, 'transactions', sep=',')

conn.commit()

TypeError: expected str, bytes or os.PathLike object, not DataFrame

In [31]:
#insert data into tables
transactions_table_insert = ("""INSERT INTO transactions(
transaction_id,
date,
salesperson,
customer)
VALUES (%s, %s, %s, %s)
""")

In [32]:
for y, row in Transactions_data.iterrows():
    cur.execute(transactions_table_insert, list(row))
conn.commit()

In [35]:
print(y)

1048574


In [42]:
count_row = Data.shape[0]
count_row

1048575

In [45]:
#second method with creating another csv with data adjusted to the table
with open('sales_data.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'sales', sep=',')
conn.commit()